In [1]:
a = 1

In [2]:
from google.protobuf import text_format
from ffn.inference import inference
from ffn.inference import inference_pb2

In [3]:
config = '''image {
  hdf5: "third_party/neuroproof_examples/validation_sample/grayscale_maps.h5:raw"
}
image_mean: 128
image_stddev: 33
checkpoint_interval: 1800
seed_policy: "PolicyPeaks"
model_checkpoint_path: "gs://arokem-retina-ffn/models/fib25/model.ckpt-27465036"
model_name: "convstack_3d.ConvStack3DFFNModel"
model_args: "{\\"depth\\": 12, \\"fov_size\\": [33, 33, 33], \\"deltas\\": [8, 8, 8]}"
segmentation_output_dir: "results/fib25/validation"
inference_options {
  init_activation: 0.95
  pad_value: 0.05
  move_threshold: 0.9
  min_boundary_dist { x: 1 y: 1 z: 1}
  segment_threshold: 0.6
  min_segment_size: 1000
}'''
req = inference_pb2.InferenceRequest()
_ = text_format.Parse(config, req)

In [4]:
import os
import tensorflow as tf
from tensorflow.contrib import tpu
from tensorflow.contrib.cluster_resolver import TPUClusterResolver

In [ ]:
tpu_grpc_url = TPUClusterResolver(tpu='ffn-tpu').get_master()

runner = inference.Runner()
with tf.Session(tpu_grpc_url, config=tf.ConfigProto(log_device_placement=True)) as sess:
    runner.start(req, session=sess)
    canvas, alignment = runner.make_canvas((0, 0, 0), (250, 250, 250), keep_history=True)
    canvas.segment_all()
    runner.save_segmentation(canvas, 
                             alignment, 
                             target_path="results/fib25/validation/target.npz", 
                             prob_path="results/fib25/validation/prob.npz")

INFO:tensorflow:Restoring parameters from gs://arokem-retina-ffn/models/fib25/model.ckpt-27465036


INFO:tensorflow:Restoring parameters from gs://arokem-retina-ffn/models/fib25/model.ckpt-27465036
/home/aman_arya524/ffn/ffn/inference/inference.py:589: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  if np.any(self.segmentation[sel] > 0):
/home/aman_arya524/ffn/ffn/inference/inference.py:421: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  self.seed[[slice(s, e) for s, e in zip(start, end)]])
/home/aman_arya524/ffn/ffn/inference/inference.py:382: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[